In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn import metrics
import numpy as np
from collections import Counter
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import mat4py
import pandas as pd
import data
import models
import pickle as pkl
from tqdm import tqdm
import train
from style import *
from sklearn import decomposition
import viz

df = data.get_data()
df = df[df['valid']] # exclude test cells, short/long tracks, hotspots
feat_names = data.get_feature_names(df)

# don't use dict_feats
feat_names = [x for x in feat_names 
              if not x.startswith('sc_') 
              and not x.startswith('nmf_')
              and not x in ['center_max', 'left_max', 'right_max', 'up_max', 'down_max', 
                            'X_max_around_Y_peak', 'X_max_after_Y_peak', 'X_max_diff_after_Y_peak']
              and not x.startswith('pc_')
#               and not 'local' in x
#               and not 'X_peak' in x
#               and not 'slope' in x
#               and not x in ['fall_final', 'fall_slope', 'fall_imp', 'fall']
             ]
print(feat_names)
print('num feats', len(feat_names))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['total_displacement', 'mean_square_displacement', 'lifetime', 'lifetime_extended', 'X_max', 'X_max_extended', 'X_min', 'X_mean', 'X_std', 'X_peak_time_frac', 'slope_end', 'X_peak_last_15', 'X_peak_last_5', 'rise', 'fall', 'rise_extended', 'fall_extended', 'fall_late_extended', 'fall_final', 'rise_slope', 'fall_slope', 'rise_local_3', 'fall_local_3', 'rise_local_11', 'fall_local_11', 'max_diff', 'min_diff', 'fall_imp']
num feats 28


# loop over classifiers

In [ ]:
outcome_def = 'y_consec_thresh'
out_dir = oj('/scratch/users/vision/abc', 'mar6_3')
os.makedirs(out_dir, exist_ok=True)
feature_selection_nums = [3, 5, 7, 15] #[3, 5, 7, 12, 16]: # number of feature to select [4, 9, 11, 23, 35, 39]
for feature_selection_num in feature_selection_nums:
    for feature_selection in ['select_rf', None]: # select_lasso, select_rf, None
        if feature_selection is None and feature_selection_num > feature_selection_nums[0]: # don't do extra computation
            break
        for model_type in tqdm(['qda', 'logistic', 'svm', 'gb', 'rf', 'mlp2']): #,'gb', 'logistic', 'dt', 'svm', 'gb', 'rf', 'mlp2', 'irf']):
            for num_feats in [16, 25, len(feat_names)]: #[16, 25, len(feat_names)]: # number of total features to consider
                for balancing in ['ros']: # None, ros                        
                    for balancing_ratio in [0.8, 1, 1.2]: # positive: negative  
                        hyperparams = [0] if model_type in ['logistic', 'rf', 'gb', 'dt', 'irf', 'qda'] else [-1, 0, 1]
                        for hyperparam in hyperparams: # 0 is default
                            feats = feat_names[:num_feats]
                            out_name = f'{model_type}_{num_feats}_{feature_selection}={feature_selection_num}_{balancing}={balancing_ratio}_h={hyperparam}'
                            train.train(df, feat_names=feats, model_type=model_type, 
                                        balancing=balancing, balancing_ratio=balancing_ratio,
                                        outcome_def=outcome_def,
                                        feature_selection=feature_selection,
                                        feature_selection_num=feature_selection_num,
                                        hyperparam=hyperparam,
                                        out_name=f'{out_dir}/{out_name}.pkl')

 67%|██████▋   | 4/6 [01:27<00:40, 20.03s/it]

# dim reduction with classifier

In [3]:
# feats_to_reduce = feat_names[:11] 
feats_to_reduce = feat_names
X = df[feat_names]
X = (X - X.mean()) / X.std()

**look at pcs**

In [ ]:
# perform pca
pca = decomposition.PCA(whiten=True)
# pca = decomposition.SparsePCA()
# pca = decomposition.NMF()
pca.fit(X)
plt.figure(figsize=(6, 9), dpi=200)
viz.plot_pcs(pca, X)

In [ ]:
plt.figure(dpi=200)
ys_neg = df[outcome_def].values == 0
plt.plot(df['pc_0'][ys_neg], df['pc_1'][ys_neg], 'o', 
         color=cr, alpha=0.3, markeredgewidth=0, ms=2) #
plt.plot(df['pc_0'][~ys_neg], df['pc_1'][~ys_neg], 'o', 
         color=cb, alpha=0.3, markeredgewidth=0, ms=2)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()